# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
document = ET.parse( './data/mondial_database.xml' )
root=document.getroot()

In [7]:

import pandas as pd
import numpy as np

# Put names of all countries, population, and infant_mortality stats in a dictionary
country_dict = {}
for element in document.iterfind('country'):
    for population in element.iterfind('population'):
        try:
             country_dict[element.find('name').text] = [float(element.find('infant_mortality').text)]
        except: 
            pass
country_df = pd.DataFrame.from_dict(country_dict, orient='index') 
country_df.reset_index(drop=False, inplace=True)
country_df.columns = ['Country', 'Infant_Mortality']

country_df.sort_values(by='Infant_Mortality').head(10)


,Country,Infant_Mortality
34,Monaco,1.81
210,Japan,2.13
71,Norway,2.48
64,Bermuda,2.48
76,Singapore,2.53
106,Sweden,2.60
55,Czech Republic,2.63
143,Hong Kong,2.73
52,Macao,3.13
189,Iceland,3.15


In [8]:
city_dict = {}
for element in document.iterfind('country'):
    for city in element.iterfind('city'):
        try:
             city_dict[city.find('name').text] = int(city.find('population').text)
        except: 
            pass
city

<Element 'city' at 0x7f5668885590>

In [9]:
city_population={}
for country in document.findall('country'):
    for city in country.findall('city'):
        try:
            city_population[city.find('name').text] = int(city.find('population').text)
        except:
            pass
city_population
city_df=pd.DataFrame.from_dict(city_population,orient='index')
city_df.reset_index(drop=False,inplace=True)
city_df.columns=['City','population']
city_df.sort_values('population',ascending=False).head(10)

,City,population
318,Seoul,10229262
45,Hong Kong,7055071
376,Al Qahirah,6053000
94,Bangkok,5876000
8,Ho Chi Minh,3924435
335,Busan,3813814
75,New Taipei,3722082
230,Hanoi,3056146
287,Al Iskandariyah,2917000
102,Taipei,2626138


In [10]:
ethnic_population={}
for country in document.iterfind('country'):
    for ethnic in country.iterfind('ethnicgroup'):
        try:
            population_percentage=ethnic.attrib['percentage']
            name=ethnic.text
            for population in country.findall('population'):
                year=population.attrib['year']
                if population.attrib['year']>=year:
                    total_population=population.text
            final_population=float(total_population)*float(population_percentage)/100
            try:
                ethnic_population[name]+=final_population
            except:
                ethnic_population[name]=final_population
        except:
            pass
def make_final_table(table,c1,c2):
    p_array=[]
    name_array=[]
    for items in ethnic_population:
        name_array.append(items)
        p_array.append(ethnic_population[items])
    f_table={c1:name_array,c2:p_array}
    return f_table
new_df=pd.DataFrame(make_final_table(ethnic_population,'name','population'))
new_df.sort_values('population',ascending=False).head(10)

,name,population
93,Han Chinese,1.245059e+09
103,Indo-Aryan,8.718156e+08
16,European,4.948722e+08
124,African,3.183251e+08
178,Dravidian,3.027137e+08
136,Mestizo,1.577344e+08
183,Bengali,1.467769e+08
179,Russian,1.318570e+08
269,Japanese,1.265342e+08
172,Malay,1.219936e+08


In [107]:
river1=[]
group=[]
for river in root.findall('river'):
    try:
        group = [river.attrib['country'], river.find('name').text, int(river.find('length').text)]
        river1.append(group)
    except:
        pass
columns=['car_code','name','length']
river_df=pd.DataFrame(columns=columns)
river_df = river_df.append(pd.DataFrame(river1, columns=columns), ignore_index=True)
river_df.sort_values('length',ascending=False).head(1)

,car_code,name,length
170,CO BR PE,Amazonas,6448


In [119]:
names_id={}
for country in document.findall('country'):
    name=country.find('name').text
    id1=country.attrib['car_code']
    names_id[name]=id1
name_df=pd.DataFrame.from_dict(names_id,orient='index')
name_df.reset_index(drop=False,inplace=True)
name_df.columns=['name','car_code']
name_df[name_df.car_code=='BR'],name_df[name_df.car_code=='CO'],name_df[name_df.car_code=='PE']

(     name car_code
 1  Brazil       BR,          name car_code
 181  Colombia       CO,     name car_code
 73  Peru       PE)

'the longest river is the amazons and it is located on Brazil, Peru, Colombia'

In [137]:
river1=[]
group=[]#I am just going to copy paste the same thing I did last time
for river in root.findall('lake'):
    try:
        group = [river.attrib['country'], river.find('name').text, int(river.find('area').text)]
        river1.append(group)
    except:
        pass
columns=['car_code','name','length']
river_df1=pd.DataFrame(columns=columns)
river_df1 = river_df.append(pd.DataFrame(river1, columns=columns), ignore_index=True)
river_df1.sort_values('length',ascending=False).head(1)

,car_code,name,length
185,R AZ KAZ IR TM,Caspian Sea,386400


In [143]:
name_df[name_df.car_code=='R'],name_df[name_df.car_code=='AZ'],name_df[name_df.car_code=='KAZ'],name_df[name_df.car_code=='IR'],name_df[name_df.car_code=='TM']

(       name car_code
 106  Russia        R,           name car_code
 51  Azerbaijan       AZ,            name car_code
 213  Kazakhstan      KAZ,      name car_code
 141  Iran       IR,            name car_code
 3  Turkmenistan       TM)

The longest lake is the caspian sea and it is located in Russia, Azerbajan, Kazakhastan, Iran and Turkemenistan

In [134]:
river1=[]
group=[]#I am just going to copy paste the same thing I did last time
for river in root.findall('airport'):
    try:
        group = [river.attrib['country'], river.find('name').text, int(river.find('elevation').text)]
        river1.append(group)
    except:
        pass
columns=['car_code','name','length']
river_df2=pd.DataFrame(columns=columns)
river_df2 = river_df.append(pd.DataFrame(river1, columns=columns), ignore_index=True)
river_df2.sort_values('length',ascending=False).head(1)

,car_code,name,length
221,BOL,El Alto Intl,4063


In [136]:
airport_df=river_df2.merge(name_df,how='left',on='car_code')
airport_df.sort_values('length',ascending=False).head(1)

,car_code,name_x,length,name_y
221,BOL,El Alto Intl,4063,Bolivia
